## Convective area and MCS identification

In [1]:
#import pyart
#import cv2
import xarray
from scipy.ndimage import measurements
%pylab inline
import warnings
warnings.filterwarnings("ignore")

/home/rjackson/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rjackson/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Populating the interactive namespace from numpy and matplotlib


In [ ]:
conv_strat_path = 'C://Users/rjackson/Documents/data/cpol_conv_strat200601200050.nc'
cpol_grid_path = 'C://Users/rjackson/Documents/data/CPOL_GRID.20060120.005008.100km.nc'

In [ ]:
cstrat_dset = xarray.open_dataset(conv_strat_path)
cstrat_dset

In [ ]:
cstrat_dset.strat_conv.plot()

In [ ]:
conv_regions = np.ma.masked_where(cstrat_dset.strat_conv.values < 2, cstrat_dset.strat_conv.values)
conv_regions[conv_regions.mask] = 0
labels = measurements.label(conv_regions)
print(labels[0].shape)
plt.pcolormesh(labels[0])
plt.colorbar()

In [ ]:
areas = np.zeros(labels[0].max()-1)
ones_array = np.ones(labels[0].shape)
for i in range(1, labels[0].max()):
    areas[i-1] = np.sum(ones_array[labels[0] == i])
print(areas)

In [ ]:
pyart_grid = pyart.io.read_grid(cpol_grid_path)
z = pyart_grid.fields['corrected_reflectivity']['data']
z_15km = z[2,:,:]
pfs = z_15km
pfs[z_15km < 15] = 0
pfs[z_15km >= 15] = 10
label_pfs = measurements.label(pfs) 
im = plt.pcolormesh(label_pfs[0], cmap='coolwarm')
ellipse_list = []
for the_labels in range(1, np.max(label_pfs[0])):
    img = np.where(label_pfs[0] == the_labels, np.ones(label_pfs[0].shape, dtype='uint8'), 
                  np.zeros(label_pfs[0].shape, dtype='uint8'))
    if(img.sum() > 30):
        ret,thresh = cv2.threshold(img,0.9,1.1,0)
        im2, contours,hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        ellipse = cv2.fitEllipse(cnt)
        ellipse_list.append(ellipse)
num_mcs = 0    
for ell in ellipse_list:
    cv2.ellipse(label_pfs[0],ell,(255,255,255),2)
    if(ell[1][0] > 100 or ell[1][1] > 100):
        num_mcs += 1
print('# of MCSs:' + str(num_mcs))

In [ ]:
print(ell[])

In [2]:
mcs_data_path = '/lcrc/group/earthscience/rjackson/data/mcs_count/*.nc'
mcs_dataset = xarray.open_mfdataset(mcs_data_path)
mcs_dataset

<xarray.Dataset>
Dimensions:     (time: 201882)
Coordinates:
  * time        (time) datetime64[ns] 1998-12-06T06:30:02 ...
Data variables:
    mcs_count   (time) float64 dask.array<shape=(201882,), chunksize=(0,)>
    dros_class  (time) float64 dask.array<shape=(201882,), chunksize=(0,)>
    mjo_index   (time) float64 dask.array<shape=(201882,), chunksize=(0,)>
Attributes:
    long_name:  Number of MCSes identified.

In [5]:
dros_index = mcs_dataset.dros_class.values
mcs_count = mcs_dataset.mcs_count.values
mjo_index = mcs_dataset.mjo_index.values

time = mcs_dataset.time
dts = [dt64_to_dt(x) for x in time]
hours = np.array([x.hour for x in dts])+9.5
hours[hours >= 24] = hours[hours >= 24] - 24



0.12324241910892766
0.19086262553802008
0.24963361016121152
0.3638314176245211


In [4]:
from datetime import datetime
def dt64_to_dt(dt):
    ts = (dt - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    return datetime.utcfromtimestamp(ts)

In [8]:
day = np.logical_and(hours >= 6, hours <= 19)
night = ~day
break_no_mjo = np.where(np.logical_and.reduce((dros_index == 0, np.logical_or(mjo_index < 4, mjo_index == 8))))
break_mjo = np.where(np.logical_and.reduce((dros_index == 0, ~np.logical_or(mjo_index < 4, mjo_index == 8))))                        
monsoon_no_mjo = np.where(np.logical_and.reduce((dros_index == 1, np.logical_or(mjo_index < 4, mjo_index == 8))))
monsoon_mjo = np.where(np.logical_and.reduce((dros_index == 1, ~np.logical_or(mjo_index < 4, mjo_index == 8))))                        
print(np.sum(mcs_count[break_no_mjo])/len(break_no_mjo[0]))
print(np.sum(mcs_count[break_mjo])/len(break_mjo[0]))
print(np.sum(mcs_count[monsoon_no_mjo])/len(monsoon_no_mjo[0]))
print(np.sum(mcs_count[monsoon_mjo])/len(monsoon_mjo[0]))


0.12897656082877487
0.20930373165010036
0.19789473684210526
0.33498145859085293
